In [7]:
""" This is the main script for eap ml project, which plays with all ml methods, applicable for cpcrsp analysis
Created on Sat Feb 12 15:19:54 2022
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import random
import gc, time
import dill
import statsmodels.api as sm
from sklearn.svm import SVR
from tensorflow.keras.callbacks import EarlyStopping


from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNetCV
from sklearn import svm
from sklearn.decomposition import PCA, KernelPCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score

from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

plt.style.use('seaborn-white')
warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', 40)
gc.enable()

In [9]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2022-04-10 22:41:57.364812: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 22:41:57.365471: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 22:41:57.365862: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 22:41:57.366322: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 22:41:57.366708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [10]:
config = tf.ConfigProto()

with tf.device('/cpu:0'):
    random_image_cpu = tf.random_normal((100, 100, 100, 3))
    net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
    net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
    random_image_gpu = tf.random_normal((100, 100, 100, 3))
    net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
    net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

cpu = lambda : sess.run(net_cpu)
gpu = lambda : sess.run(net_gpu)
  
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [2]:
path = '../input/mleap-data/'

s_train_X = pd.read_csv(path + 's_train_X.csv')
s_train_y = pd.read_csv(path + 's_train_y.csv')
s_test_X = pd.read_csv(path + 's_test_X.csv')
s_test_y = pd.read_csv(path + 's_test_y.csv')
sp_train_X = pd.read_csv(path + 'sp_train_X.csv')
lp_train_X = pd.read_csv(path + 'lp_train_X.csv')
sp_test_X = pd.read_csv(path + 'sp_test_X.csv')
lp_test_X = pd.read_csv(path + 'lp_test_X.csv')

s_train_X.drop(columns='Unnamed: 0', inplace=True)
s_train_y.drop(columns='Unnamed: 0', inplace=True)
s_test_X.drop(columns='Unnamed: 0', inplace=True)
sp_train_X.drop(columns='Unnamed: 0', inplace=True)
lp_train_X.drop(columns='Unnamed: 0', inplace=True)
lp_test_X.drop(columns='Unnamed: 0', inplace=True)

s_train_X = np.array(s_train_X)
s_train_y = np.array(s_train_y)
s_test_X = np.array(s_test_X)
s_test_y = np.array(s_test_y)
sp_train_X = np.array(sp_train_X)
lp_train_X = np.array(lp_train_X)
sp_test_X = np.array(sp_test_X)
lp_test_X = np.array(lp_test_X)

print(s_train_X.shape)
print(s_train_y.shape)
print(s_test_X.shape)
print(s_test_y.shape)
print(sp_train_X.shape)
print(lp_train_X.shape)
print(sp_test_X.shape)
print(lp_test_X.shape)

(1257579, 27)
(1257579, 1)
(100000, 27)
(100000, 1)
(1257579, 10)
(1257579, 50)
(100000, 10)
(100000, 50)


In [3]:
model_ann5_s = Sequential([
    BatchNormalization(input_shape=(27,)),
    Dense(256, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(16, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(1)])

print(model_ann5_s.count_params())

159757


2022-04-10 22:34:24.009133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 22:34:24.116103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 22:34:24.117285: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 22:34:24.118784: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
time1 = time.time()

early_stopping20 = EarlyStopping(patience=20)
early_stopping15 = EarlyStopping(patience=15)
early_stopping10 = EarlyStopping(patience=10)
early_stopping5 = EarlyStopping(patience=5)


model_ann5_s.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
history = model_ann5_s.fit(s_train_X, s_train_y, validation_data=(s_test_X, s_test_y), 
                         batch_size=128, epochs=5, verbose=2, callbacks=[early_stopping5])
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

ann_s = [r2_score(s_train_y, model_ann5_s.predict(s_train_X)), 
       r2_score(s_test_y, model_ann5_s.predict(s_test_X))]

print('time is ', time.time()-time1)

Epoch 1/5
9825/9825 - 56s - loss: 0.9948 - mean_squared_error: 0.9948 - val_loss: 1.1291 - val_mean_squared_error: 1.1291
Epoch 2/5
9825/9825 - 55s - loss: 0.9938 - mean_squared_error: 0.9938 - val_loss: 1.1295 - val_mean_squared_error: 1.1295
Epoch 3/5


KeyboardInterrupt: 

In [ ]:
ann_s 

In [ ]:
import shap
shap.initjs()

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model_ann5_s)
shap_values = explainer.shap_values(s_train_X)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], s_train_X.iloc[0,:])

shap.summary_plot(shap_values, s_train_X, plot_type="bar")

In [ ]:
explainer = shap.GradientExplainer(model_ann5_s, s_train_X)
#shap_values = explainer.shap_values(s_test_X)

In [ ]:
shap_values = explainer.shap_values(s_test_X)

In [ ]:
shap.summary_plot(shap_values[0], plot_type = 'bar', feature_names = test_data.columns)

In [ ]:
shap.summary_plot(shap_values[0], plot_type = 'bar', feature_names = test_data.columns)

In [ ]:
import pickle

with open("ann_results_2", "wb") as fp:   #Pickling
    pickle.dump(ann_results, fp)

In [ ]:
temp = pd.DataFrame(ann_results)
temp.to_csv('ann_results_2.csv')

In [ ]:
!pip install tensorflow==2.5.0
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

print (tf.__version__)

In [ ]:
print (tf.__version__)

In [ ]:
# import the modules we'll need
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [ ]:
create_download_link(temp)
